# Benchmark Vision Models on CIFake Dataset

This notebook benchmarks convolutional and transformer-based networks defined in `vision_models.py` on the [CIFake dataset](https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images).


In [ ]:
!pip install -q kaggle torchvision

In [ ]:
import os
from pathlib import Path
import numpy as np
import torch
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from pipelines_torch.benchmark import BenchmarkRunner
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from ml_pipeline.pipelines_torch.vision_models import get_model, MODEL_REGISTRY

DATA_DIR = Path('cifake_data')

In [ ]:
# Requires Kaggle API credentials available as environment variables
# KAGGLE_USERNAME and KAGGLE_KEY. See https://www.kaggle.com/docs/api.
if not DATA_DIR.exists():
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    !kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images -p $DATA_DIR --unzip

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

train_ds = datasets.ImageFolder(DATA_DIR / 'train', transform=transform)
X = torch.stack([img for img, _ in train_ds]).numpy()
y = np.array(train_ds.targets)
val_ds = datasets.ImageFolder(DATA_DIR / 'test', transform=transform)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)

class_names = train_ds.classes
num_classes = len(class_names)

In [ ]:
from torch import nn, optim

def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, running_acc = 0.0, 0.0
    for x, y in loader:
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.size(0)
        running_acc += (preds.argmax(1) == y).sum().item()
    return running_loss / len(loader.dataset), running_acc / len(loader.dataset)


def evaluate(model, loader, criterion):
    model.eval()
    loss, acc = 0.0, 0.0
    with torch.no_grad():
        for x, y in loader:
            preds = model(x)
            loss += criterion(preds, y).item() * x.size(0)
            acc += (preds.argmax(1) == y).sum().item()
    return loss / len(loader.dataset), acc / len(loader.dataset)

In [ ]:
results = {}
for name in MODEL_REGISTRY:
    model = get_model(name, num_classes=num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    results[name] = {'train_acc': train_acc, 'val_acc': val_acc}
    print(name, results[name])

results